In [1]:
import json 
import pandas as pd
from datetime import datetime

with open('schlepps_2024.json') as f:
    raw_data = json.load(f)

weekdays_de = {
    0: "montag",
    1: "dienstag",
    2: "mittwoch",
    3: "donnerstag",
    4: "freitag",
    5: "samstag",
    6: "sonntag"
}

# Flugtage
print("Flugtage, total:", len(raw_data))
flat_data = []
for k in raw_data:
    d = datetime.fromisoformat(k)
    #print(k,d.month, d.weekday())
    assert(weekdays_de[d.weekday()]==raw_data[k]['weekday'])
    d_schlepps, dienst = raw_data[k]['wf'], raw_data[k]['dienst'],
    for wf in d_schlepps:
        row = [k, 
                wf.split(' ')[0],
                None if ' ' not in wf else wf.split(' ')[1],
                d_schlepps[wf], 
                d.month in range(4,10), 
                d.weekday() in [5,6],
                dienst !="" and dienst in wf, # should include (ewf)
                d.weekday() == 6
                ]

        flat_data.append(row)

df = pd.DataFrame(flat_data, columns=['datum','wf','ewf','schlepps','hauptsaison','wochenende','dienst','sonntag'])

#print(df.tail(10))
print("WF, total", len(df['wf'].unique()))
print("Schlepps, total", df['schlepps'].sum())
print("Schlepps, hauptsaison", df[df['hauptsaison']]['schlepps'].sum())
print("Schlepps, hauptsaison+wochenende", df[df['hauptsaison'] & df['wochenende']]['schlepps'].sum())
print("Schlepps, dienst", df[df['dienst']]['schlepps'].sum())
print("Schlepps, dienst Sonntag", df[df['dienst'] & df['sonntag'] ]['schlepps'].sum())

df_by_wf = df[['wf','schlepps']].groupby(by="wf").sum().sort_values(by='schlepps',ascending=False).copy(deep=True)
df_by_ewf = df[['ewf','schlepps']].groupby(by="ewf").sum().sort_values(by='schlepps',ascending=False).copy(deep=True)

print("Schlepps, Durchschnitt pro WF", round(df_by_wf['schlepps'].mean(),1))

print(df_by_ewf.head())

df_by_wf



Flugtage, total: 25
WF, total 18
Schlepps, total 628
Schlepps, hauptsaison 532
Schlepps, hauptsaison+wochenende 375
Schlepps, dienst 116
Schlepps, dienst Sonntag 63
Schlepps, Durchschnitt pro WF 34.9
          schlepps
ewf               
(Michi)         72
(Helmut)        46


,schlepps
wf,
Akos,94
Thomas,81
Utz,75
Matthias,73
Michi,64
Martin,36
Wolfgang,36
Helmut,35
Hristo,30


In [18]:

print(df[df['hauptsaison'] & (~df['wochenende'])]['schlepps'].agg(['sum']))

print(df[df['datum'].isin(['2024-05-11','2024-05-25','2024-05-26'],)]['schlepps'].agg(['sum']))

df[df['dienst'] & df['sonntag'] ][['datum','schlepps']]

df[~df['dienst'] & df['sonntag'] ]['schlepps'].agg(['sum'])

sum    157
Name: schlepps, dtype: int64
sum    124
Name: schlepps, dtype: int64


sum    150
Name: schlepps, dtype: int64